## 4.6. 暂退法（Dropout）

希望模型深度挖掘特征，即将其权重分散到许多特征中， 而不是过于依赖少数潜在的虚假关联。

### 4.6.1. 重新审视过拟合

泛化性和灵活性之间的这种基本权衡被描述为偏差-方差权衡（bias-variance tradeoff）

线性模型有很高的偏差：它们**只能表示一小类函数**。然而这些模型的方差很低：它们在**不同的随机数据**样本上可以得出**相似**的结果

深度神经网络位于偏差-方差谱的另一端.神经网络并不局限于单独查看每个特征，而是**学习特征之间的交互**

**深度网络的泛化性质令人费解，而这种泛化性质的数学基础仍然是悬而未决的研究问题。**

### 4.6.2. 扰动的稳健性

“好”的预测模型能在未知的数据上有很好的表现： 经典泛化理论认为，为了缩小训练和测试性能之间的差距，应该以简单的模型为目标。*简单性以较小维度的形式展现*

简单性的另一个角度是**平滑性**，即**函数不应该对其输入的微小变化敏感**。

1995年，克里斯托弗·毕晓普证明了 具有输入噪声的训练等价于Tikhonov正则化,用数学证实了“要求函数光滑”和“要求函数对输入的随机噪声具有适应性”之间的联系。[Bishop, 1995](https://doi.org/10.1162/neco.1995.7.1.108)
在2014年，斯里瓦斯塔瓦等人 ([Srivastava et al., 2014](https://www.researchgate.net/publication/286794765_Dropout_A_Simple_Way_to_Prevent_Neural_Networks_from_Overfitting)) 就如何将毕晓普的想法应用于网络的内部层提出了一个想法： 在训练过程中，他们建议在计算后续层之前向网络的每一层注入噪声。 因为当训练一个有多层的深层网络时，注入噪声只会在输入-输出映射上增强平滑性。

**暂退法（dropout）**：前向传播中，在每一层的输入中增加噪声。
从表面上看是在训练过程中丢弃（drop out）一些神经元。 在整个训练过程的每一次迭代中，标准暂退法包括在计算下一层之前将当前层中的一些节点置零。

**如何注入这种噪声**：一种无偏向（unbiased）的方式注入噪声。以概率$p$将节点活性值$h$用$h'$代替，其中$\mathrm{P}(h'=0)=p,\ \mathrm{P}(h'=\frac{h}{1-p})=1-p$。计算期望值可得：$E[h']=h$，所以此种注入噪声方式可以保证其期望值不发生改变。

**根据指定的暂退概率随机丢弃上一层的输出（相当于下一层的输入）**

### 4.6.3.实践中的暂退法

通常，我们**在测试时不用暂退法**。 给定一个训练好的模型和一个新的样本，我们不会丢弃任何节点，因此不需要标准化。

例外：一些研究人员在测试时使用暂退法， 用于估计神经网络预测的“不确定性”： 如果通过许多不同的暂退法遮盖后得到的预测结果都是一致的，那么我们可以说网络发挥更稳定。

In [1]:
import torch
from torch import nn

def dropout_layer(X, dropout):
    assert 0 <= dropout <= 1
    # 在本情况中，所有元素都被丢弃
    if dropout == 1:
        return torch.zeros_like(X)
    # 在本情况中，所有元素都被保留
    if dropout == 0:
        return X
    mask = (torch.rand(X.shape) > dropout).float() # 实现dropout的关键
    return mask * X / (1.0 - dropout)

In [2]:
X= torch.arange(16, dtype = torch.float32).reshape((2, 8))
print(X)
print(dropout_layer(X, 0.))
print(dropout_layer(X, 0.5))
print(dropout_layer(X, 1.))

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])
tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])
tensor([[ 0.,  0.,  0.,  0.,  0.,  0., 12.,  0.],
        [ 0.,  0., 20.,  0.,  0.,  0., 28., 30.]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


### 4.6.6. 小结

暂退法在前向传播过程中，计算每一内部层的同时丢弃一些神经元。

暂退法可以避免过拟合，它通常与控制权重向量的维数和大小结合使用的。

暂退法$h$活性值
替换为$h$有期望值
的随机变量。

暂退法仅在训练期间使用。